<a href="https://colab.research.google.com/github/thiagomantuani/meli-data-challenge-2021/blob/main/1_join_datasets_meli_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm
!pip install jsonlines

In [ ]:
from google.colab                    import drive   
import pandas                        as pd
import numpy                         as np
import os
import gc
import tqdm
import jsonlines

In [ ]:
drive.mount('/gdrive')
path = '/gdrive/My Drive/DataSets/meli2021/'
os.chdir(path)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def load_jsonlines(filename):    
    rv = []
    for obj in tqdm.tqdm(jsonlines.open(filename)):
        rv.append(obj)
    return rv

In [ ]:
df = pd.read_parquet(path+'/train_data.parquet')
df = reduce_mem_usage(df)

Mem. usage decreased to 12244.10 Mb (-373.5% reduction)


In [ ]:
item_metadata = load_jsonlines(path+'/items_static_metadata_full.jl')

660916it [00:06, 102664.22it/s]


In [ ]:
df_itens = pd.DataFrame(item_metadata)
df_itens.index = df_itens.sku
df_itens.drop(columns=['sku'],inplace=True)

In [ ]:
df_full = df.join(df_itens,on='sku',how='left')

In [ ]:
del df
del df_itens
del item_metadata
gc.collect()

135

In [ ]:
df_full = reduce_mem_usage(df_full)

Mem. usage decreased to 23660.35 Mb (-558.8% reduction)


In [ ]:
df_full = df_full.drop(columns=['item_title','product_id','product_family_id','item_id'],axis=1)

In [ ]:
df_full.head()

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,item_domain_id,site_id
0,464801,2021-02-01,0,156.779999,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
1,464801,2021-02-02,0,156.779999,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
2,464801,2021-02-03,0,156.779999,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
3,464801,2021-02-04,0,156.779999,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
4,464801,2021-02-05,1,156.779999,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB


In [ ]:
df_full.shape

(37660279, 11)

In [ ]:
df_full.set_index('sku',inplace=True)

In [ ]:
%%time
df_full.to_csv('train_data_full.csv.gz',compression='gzip')

CPU times: user 5min 21s, sys: 1.79 s, total: 5min 23s
Wall time: 5min 24s


In [ ]:
df2 = pd.read_csv(path+'/train_data_full.csv.gz',compression="gzip",index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df2.head()

,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,item_domain_id,site_id
sku,,,,,,,,,,
464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB


## References

https://www.kaggle.com/rinnqd/reduce-memory-usage

https://ml-challenge.mercadolibre.com/recipe/2021/baseline